In [1]:
import numpy as np
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
from mindspore import Parameter, Tensor

In [2]:
def make_batch(seq_data, word_dict,vocab_size):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word_dict[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word_dict[seq[-1]] # 'e' is target
        input_batch.append(np.eye(vocab_size)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [3]:
class TextLSTM(nn.Cell):
    def __init__(self, batch_size, vocab_size, hidden_size):
        super(TextLSTM,self).__init__()
        self.lstm = nn.LSTM(input_size=vocab_size, hidden_size=hidden_size)
        self.W = nn.Dense(hidden_size, vocab_size, has_bias=False)
        self.b = Parameter(Tensor(np.ones(vocab_size), mindspore.float32), 'b')
        
        self.n_steps = n_steps
        self.hidden_state = Tensor(np.zeros((1, batch_size, hidden_size)).astype(np.float32))  # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        self.cell_state = Tensor(np.zeros((1, batch_size, hidden_size)).astype(np.float32))

    def construct(self, X):
        input = X.transpose((1, 0, 2))  
        outputs, (_, _) = self.lstm(input, (self.hidden_state, self.cell_state))
        outputs = outputs[-1] 
        model = self.W(outputs) + self.b  
        return model

In [4]:
n_steps = 3 
hidden_size = 128 

char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz']
word_dict = {n: i for i, n in enumerate(char_arr)}
number_dict = {i: w for i, w in enumerate(char_arr)}
vocab_size = len(word_dict)  

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

In [5]:
input_batch, target_batch = make_batch(seq_data, word_dict, vocab_size)
input_batch = Tensor(input_batch, mindspore.float32)
target_batch = Tensor(target_batch, mindspore.int32)

In [6]:
batch_size = len(input_batch)

In [7]:
model = TextLSTM(batch_size, vocab_size, hidden_size)

In [8]:
criterion = nn.SoftmaxCrossEntropyWithLogits(sparse=True,reduction='mean')
optimizer = nn.Adam(model.trainable_params(), learning_rate=0.001)

In [9]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE)

net_with_criterion = nn.WithLossCell(model, criterion)
train_network = nn.TrainOneStepCell(net_with_criterion, optimizer)
train_network.set_train()

TrainOneStepCell<
  (network): WithLossCell<
    (_backbone): TextLSTM<
      (lstm): LSTM<
        (rnn): _DynamicLSTMCPUGPU<>
        (reverse): _Reverse<>
        (reverse_sequence): _ReverseSequence<>
        (dropout_op): Dropout<keep_prob=1.0>
        >
      (W): Dense<input_channels=128, output_channels=26>
      >
    (_loss_fn): SoftmaxCrossEntropyWithLogits<>
    >
  (optimizer): Adam<>
  >

In [10]:
# Training
epoch = 1000
for step in range(epoch):
    loss = train_network(input_batch, target_batch)
    if (step + 1) % 100 == 0:
        print('Epoch:', '%04d' % (step + 1), 'cost = ', '{:.6f}'.format(loss.asnumpy()))

[WARNING] DEBUG(16198,0x10681c600,python):2022-01-23-19:55:07.133.408 [mindspore/ccsrc/debug/debugger/debugger.cc:95] Debugger] Not enabling debugger. Debugger does not support CPU.


Epoch: 0100 cost =  1.162239
Epoch: 0200 cost =  0.185100
Epoch: 0300 cost =  0.034191
Epoch: 0400 cost =  0.011772
Epoch: 0500 cost =  0.005911
Epoch: 0600 cost =  0.003599
Epoch: 0700 cost =  0.002443
Epoch: 0800 cost =  0.001777
Epoch: 0900 cost =  0.001355
Epoch: 1000 cost =  0.001069


In [11]:
inputs = [sen[:3] for sen in seq_data]

In [12]:
predict = model(input_batch).asnumpy().argmax(axis=1)
print(inputs, '->', [number_dict[n.item()] for n in predict.squeeze()])

['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
